# PRODUCTION phase: About this notebook
- Purpose: Creates 1 of the 2 PIPELINES
    - `2a) training pipeline:` TRAINS a model with Azure AutoML and with AZURE compute cluster and calculates test_set scoring, automatically compares if newly trained model is better.
    
## DETAILS - about this notebook and the 2a pipeline, generated
- 1) Initiate ESMLPipelineFactory:
- 2) `AUTO-GENERATE code: a snapshot folder` via ESML, that generates Python scripts and the `ESML runtime`
    - 2_A_aml_pipeline\4_inference\batch\\`M11`
        - Edit the feature engineerin files if needed
            - 2_A_aml_pipeline\4_inference\batch\\`M11\your_code\your_custom_code.py`
            - `your_custom_code.py` is referenced from all the `in_2_silver_...` files such as: 2_A_aml_pipeline\4_inference\batch\M11\\`in2silver_ds01_diabetes.py` and `silver_merged_2_gold`
        - Edit the AutoML train file, you need to add some configuration. See instructions in notebook cells below.
            -2_A_aml_pipeline\4_inference\batch\\`M11\train_post_automl_step.py`
- 3) `BUILDS the pipeline` of certain (IN_2_GOLD_TRAIN_AUTOML)
    - An `Azure machine learning pipeline` with steps will be automatically genereated, based on your `lake_settings.json` dataset array.
    - It is a `training pipeline` of ESML type `IN_2_GOLD_TRAIN_AUTOML`
- 4) `EXECUTES the pipeline` (smoke testing purpose - see that it works...)
    - 4a) The below happens in the pipeline steps:Training pipeline: (`IN_2_GOLD_TRAIN_AUTOML`) steps:
        - Feature engineering of each in-data - via `IN_2_SILVER` steps.
        - Merges all SILVERS to `GOLD`
        - Splits the `GOLD` to 3 buckets: `GOLD_TRAIN, GOLD_VALIDATE, GOLD_TEST`
        - Trains model
        - Registers the newly trained model, tags it as `newly_trained`
        - Calculates test_set scoring with the `ESMLTestescoringFactory`
        - `INNER LOOP MLOps:` Compares in current environment `DEV` if model should be promoted, based on `test_set_scoring`
        - `OUTER LOOP MLOps:` Compares in next environment `TEST` if model should be promoted, based on `test_set_scoring` 
            - E.g. compares best model in `DEV` with the leading model in `TEST`
- 5) PUBLISH the pipeline
    - Purpose: Now when the pipeline is `smoke tested`, we can publish is, to get a `pipeline_id to use in Azure Data factory`
    - We want to PRINT the `pipeline ID` after publish also, for easy access to use in `Azure data factory` for retraining on new data continously (DataOps & MLOps)
- DONE.
    

Note:This notebook is called: `M11_v143_esml_regression_batch_train_automl.ipynb` in the notebook_templates folder
 

# TODO for you: CONFIGURATION
- 1) Change `p.active_model=11` to correct model number `1` if your model has that number.
    - See  [lake_settings.json](./settings/project_specific/model/lake_settings.json) to find YOUR model number.
- 2) After you run the cell [2) AUTO-GENERATE code: a snapshot folder](#2_generate_snapshot_folder), you need to add YOUR feature engineering logic
    -  This code you probably already have, from the R&D phase, in this CUSTOMIZE cell in the notebook: [1_R&D_phase_M10_M11.ipynb](./1_quickstart/1_R&D_phase_M10_M11.ipynb)
        - You need to this code to the `your_custom_code.py` after you have genereated the snapshot folder, for it to be reachable and uploaded at pipeline creation.
        - Tip: You can CREATE A CLASS, and add static methods, e.g. `ds01_process_in2silver(dataframe1)`  in the `your_custom_code.py` 
- 3) Now you have your code in the `your_custom_code.py`, then you need to reference that code from the auto-generated pipeline-steps files such as `in2silver_ds01_diabetes.py`
    - Note: This snapshot folder will not exist, until you have run the first 2 cells in this notebook, or after this cell has run [2) AUTO-GENERATE code: a snapshot folder](#2_generate_snapshot_folder)

## 1) Initiate ESMLPipelineFactory (Always run thic CELL below)
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPipelinefactory

In [ ]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../settings/model/active/active_in_folder.json',
p.inference_mode = False
p.active_model = 11 # 10=titanic , 11=Diabetes
p_factory = ESMLPipelineFactory(p)

training_datefolder = '1000-01-01 10:35:01.243860' # Will override active_in_folder.json
p_factory.batch_pipeline_parameters[0].default_value = 0 # Will override active_in_folder.json.model.version = 0 meaning that ESML will find LATEST PROMOTED, and not use a specific Model.version. It will read data from .../inference/0/... folder
p_factory.batch_pipeline_parameters[1].default_value = training_datefolder # overrides ESMLProject.date_scoring_folder.
p_factory.describe()


# The below cells for an IN_2_GOLD_TRAIN_AUTOML pipeline will:
- 1) Generate code files
- 2) Build pipeline, ESML autoguild this, and will upload the snapshot folder together with the Azure ML pipeline.
- 3) Run the pipeline. Smoke testing, see that it works
- 4) IF it works, Publish the pipeline, or else, edit the code files or configuration, retry step 2 and 3.
- 5) Print the pipeline_id, that is essential to use from Azure Data factory 

# 2) `AUTO-GENERATE code: a snapshot folder`
<a id='2_generate_snapshot_folder'></a>

In [ ]:
## Generate CODE - then edit it to get correct environments
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT

# TRAINING (3a,4a,5a)

# 3) `BUILDS the TRANING pipeline`
- esml_pipeline_types.IN_2_GOLD_TRAIN_AUTOML
- Take note on the `esml_pipeline_types` below, of type: esml_pipeline_types.`IN_2_GOLD_TRAIN_AUTOML`

In [ ]:
## batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_TRAIN_MANUAL, same_compute_for_all=True, cpu_gpu_databricks="cpu", allow_reuse=True)

## BUILD (takes ~10-12minutes)
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_TRAIN_AUTOML)
# ...which Trains a model on data via date_folder parameters, upload the generated python scripts., and your custom code and ESML runtime, to Azure embedded in the pipeline, using Dockerized image. 

## 4a) `EXECUTES the pipeline`

### NB! Run in v1 legacy mode
- You need to have your Azure Machine Learning workspace set to `v1_legacy_mode=True`
- HOW do I know if I run v1 or v2? 
  - If you see this error message in `executionlogs.txt in Azure machine learning studio Output+logs tab on pipeline rune`, containing the word in path `backendV2` when executing pipeline (cell below this), it is not in v1 legacy mode:
     - <i>Failed to start the job for runid: 33ff1e3a-1ca7-4de0-bcee-b851cd2bb89d because of exception_type: ServiceInvocationException, error: Failure in StartSnapshotRun while calling service Execution; HttpMethod: POST; Response StatusCode: BadRequest; Exception type: Microsoft.RelInfra.Extensions.HttpRequestDetailException|-Microsoft.RelInfra.Common.Exceptions.ErrorResponseException, stack trace:    at Microsoft.Aether.EsCloud.Common.Client.ExecutionServiceClient.StartSnapshotRunAsync(String jobId, RunDefinition runDefinition, String runId, WorkspaceIdentity workspaceIdentity, String experimentName, CreatedBy createdBy) in D:\a\_work\1\s\src\aether\platform\\`backendV2`\\Clouds\ESCloud\ESCloudCommon\Client\ExecutionServiceClient.cs:line 162
   at Microsoft.Aether.EsCloud.Common.JobProcessor.StartRunAsync(EsCloudJobMetadata job) in D:\a\_work\1\s\src\aether\platform\backendV2\Clouds\ESCloud\ESCloudCommon\JobProcessor.cs:line 605
   </i>
- WHY? 
    - Azure ML SDK v2 does not yet (writing this 2022-10)support Spark jobs in pipeline, nor private endpoint.
- TODO: To set the workspace in LEGACY v1 mode run this code 1 time, in a cell: `p.ws.update(v1_legacy_mode=True)`

In [ ]:
#p.ws = p.get_workspace_from_config()
#p.ws.update(v1_legacy_mode=True) # If you happen to have a workspace in v2 mode, and want to change back to v1 legacy mode

In [ ]:
## RUN and it will train in BIG Data, since using 100% Azure compute for all steps, including SPLITTING data
pipeline_run = p_factory.execute_pipeline(batch_pipeline) # If this give ERROR message, looking at executionlogs.txt in Azure machine learning studio Output+logs tab on pipeline rune
pipeline_run.wait_for_completion(show_output=False)

# 4b) View meta data about the training run
- What DATA was used, WHEN did the training occur, etc

In [ ]:
from azureml.core import Dataset
ds_name ="{}_GOLD_TRAINED_RUNINFO".format(p.ModelAlias)
meta_ds= Dataset.get_by_name(workspace=p.ws,name=ds_name, version='latest')
meta_ds.to_pandas_dataframe().head()

# 5a) PUBLISH the TRAINING pipeline & PRINT its ID

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

# PRINT: Get info to use in Azure data factory
- `published_pipeline.id` (if private Azure ML workspace)

In [ ]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

# DONE! Next Step - Deploy model, serve your model for INFERENCING purpose:
- For INFERENCE you may need either to DEPLOY the model 
    - a) ONLINE on AKS endpoint
        - Notebook: 
    - b) BATCH SCORING on an Azure machine learning pipeline
        - Notebook: [your_root]\notebook_templates_quickstart\\`3a_PRODUCTION_phase_BATCH_INFERENCE_Pipeline.ipynb`
    - c) STREAMING using Eventhubs and Azure Databricks structured streaming
        - Notebook: TBA

- Q: `Next step in PRODUCTION phaase after the 2a and 3a or 3b notebooks are done?`
 
- 1) `DataOps+MLOps:` Go to your ESMLProjects `Azure data factory`, and use the `ESML DataOps templates` (Azure data factory templates) for `IN_2_GOLD_TRAIN` and `IN_2_GOLD_SCORING`
    - azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\adf\v1_3\PROJECT000\LakeOnly\\`STEP03_IN_2_GOLD_TRAIN_v1_3.zip`
- 2) `MLOps CI/CD` Go to the next notebook `mlops` folder, to setup `CI/CD` in Azure Devops
    - Import this in Azure devops
        azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\mlops\01_template_v14\azure-devops-build-pipeline-to-import\\`ESML-v14-project002_M11-DevTest.json`
    - Change the Azure Devops `VARIABLES` for service principle, tenant, etc.
    - Change parameters in the `inlince Azure CLI script` to correct model you want to work with, and the correct data you want to train with, or score.
        - Step `21-train_in_2_gold_train_pipeline`
        - INLINE code calls the file: `21-train_in_2_gold_train_pipeline.py`
        - INLINE parameters: `--esml_model_number 11 --esml_date_utc "1000-01-01 10:35:01.243860"`